# NMF - Non-Negative Matrix Factorization

by Sergiu Netotea, PhD, NBIS, Chalmers

TODO:
- https://nimfa.biolab.si/index.html
- `Biological data usually contain missing values. How can the matrix factorization handle it?

NMF handles missingness in several ways. By using initialization methods such as nndsvd that are friendlier towards sparse matrices is number one. Then for the optimization norm I would pick KL if you have large matrices with a lot of missingness. Rather than doing distances KL would evaluate changes in how data is distributed. Also note that you can add a regularization term to the fitness function. This does not help directly but in sparse datasets it is easy to get trapped in local optima.
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

- Is NMF used as an unsupervised clustering methods in single-cell RNA seq? If so, how does it perform, or does it provode advantages compared to other used algorithms? can you point us to any tool/implementation of NMF which is appropiate for single-cell clustering analysis?

iNMF was used for clustering in a very very recent study (that I had no time to inspect unfortunately, but worth check it out). There is code available too!
Gao, C., Liu, J., Kriebel, A.R. et al. Iterative single-cell multi-omic integration using online learning. Nat Biotechnol (2021). https://doi.org/10.1038/s41587-021-00867-x


- `Should data be normalized before coercing it into a matrix factorization model?`

Generally yes, and this generally goes before any application of machine learning to real data. But in particular it might not be the case. If you apply ML on a single omics dataset then the data usually comes in a normalized shape, something which you can inspect by plotting your samples. Even if data was normalized, and especially for integrative omics, it is worth checking if the normalization was applied uniformly, since in most cases that is not done.
However if the normalization is a big issue for your data (such as huge differences in scale) then you should opt for Tree Boosting methods (random forest, ensemble learning) or SNF, rather than matrix factorization. In matrix factorization the variables are not considered independently, instead everyting is dependent (linearly) on everything else. There are many ways to skin a cat. :)


### Recommender systems

- Basic definition: recommender systems is what we do when we buy stuff.
    - Recommend an item to a user, recommend a user to an item.
- Why is this important for a life science researcher?
    - Reccomend a multi-omic cause for a phenotipic effect, or the opposite.
    - > "Research is a derivative of a society's general drift, rarely the opposite." :) personal view, reader discretion is advised!
    - Rich guys have THE BEST data. Here are some example data tables:
        - user/likes section: users/films, users/pictures, users/groceries, users/houses, users/profiles
        - item/usage section: item/construction costs, item/delivery options, item/user rankings
        - tax avoidance/country section: ... etc
    - If recommender systems research had the same budget as multi-omics research:
        - <img src="img/budget.jpg" width="200"/>

### Recommender systems vs Systems Biology

- Each user (sample) can be described by k attributes or features. Example: How likely is it that a person suffers from a certain type of cancer?
- Each item (gene) can be described by an analagous set of k attributes or features. Example: How likely is it for the gene to be involved/co-regulated/induced etc in a certain type of cancer.
- We don't always know what these features are, how many are relevant. We learn them, or rather let the machines pick.
- Collaborative filtering: Users who share the same interest in certain kind of items will also share the same interest in some other kind of items.


In [11]:
import pandas as pd
import numpy as np

m = np.array([[0,1,0,1,2,2],
             [0,1,1,1,3,4], 
             [2,3,1,1,2,2],
             [1,1,1,0,1,1],
             [0,2,3,4,1,1],
             [0,0,0,0,1,0]])

groceries = pd.DataFrame(m, columns=['John', 'Alice', 'Mary', 'Greg', 'Peter', 'Jennifer'])
groceries.index = ['Potatoes', 'Onions', 'Fruits', 'Sweets', 'Bread', 'Coffee']
print(groceries)

          John  Alice  Mary  Greg  Peter  Jennifer
Potatoes     0      1     0     1      2         2
Onions       0      1     1     1      3         4
Fruits       2      3     1     1      2         2
Sweets       1      1     1     0      1         1
Bread        0      2     3     4      1         1
Coffee       0      0     0     0      1         0


Intuitively we can see that the users (samples) are conected to their items (genes) via a hidden scheme, that could simplify this table. The elements of such a hidden scheme we call hidden (latent) factors. Here is a possible example:

In [6]:
latent_factors = ['Fruits pickers', 'Bread eaters',  'Veggies']

### Matrix factorization

<img src="img/NMF.png" width="300"/>
[Credit: Wikipedia]

Hidden representation of a user u predicted rating for item i based on k latent factors (hidden features):
- $r_{ui} = x_u y_i^T = \sum_k{x_{uk} y_{ki}}$
- Fit function: $ L = \sum_{u,i \in S} (r_{ui} - x_u y_i^T)^2$


Methods for computing MF:

- Many methods exist, most notable being:
    - alternating least squares (ALS): Iteratively fix either $x_u$ or $y_i$. This is because in itself the MF is a non-convex optimization problem that can be turned into a quadratic optimization problem by fixing the factor matrices.
    - stochastic gradient descent (SGD): A gradient is computed by the derivative of the loss function for each variable, then the weights are updated by a factor in the opposite direction of the gradient.


- For an *ab-initio* example on how to perform general matrix factorization using 2 methods:
    - https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea

### NMF - Non-Negative Matrix Factorization

- Similar to ICA, PCA, MFA it can be classified as an unsupervised dimensionality reduction / clustering technique.
- This non-negativity also implies additivity of latent factors. This is in line with the idea that expression pattern of a gene across samples is the weighted sum of multiple metagenes. It is also the case for many intensity based 'omics measurements.

- Constraints:
$$
min~\|X-WH\|_F \\
X \ge 0, \\
W \ge 0, \\
H \ge 0,
$$

- Uses the Frobenius norm, which is the matrix equivalent of the euclidean distance. But other cost functions are possible, that also include regularization.
$$
\|X\|_F = \sqrt{\sum_i\sum_jx_{ij}^2}.
$$
- (Many algorithms exist, see below). Main solver works iteratively via ALS as:
$$\begin{align}
    W_{t+1} &= W_t^T \frac{XH_t^T}{XH_tH_t^T} \\
    H_{t+1} &= H_t \frac{W_t^TX}{W^T_tW_tX}.
\end{align}$$
- Weak convergence: it is reinitialized several times to avoid local minima, and the best result is kept.
- Optimal number of components (RSS scores for example, Silhouettes scores etc)

### NMF - multi omics usage observations

- The different omics datasets have different scales and normalities, and this will impact your results (but it also depends on assumptions, on the goal of your study).
- To bring the different datasets Frobenius norms to the same baseline one can do:
$$
X = \begin{bmatrix}
    \frac{x^{1j}}{\sum_jx^{1j}} /  \|\frac{x^{1j}}{\sum_jx^{1j}}\|_F \\
    \frac{x^{2j}}{\sum_jx^{2j}} /  \|\frac{x^{2j}}{\sum_jx^{2j}}\|_F \\
    \vdots \\
    \frac{x^{Nj}}{\sum_jx^{Nj}} /  \|\frac{x^{Nj}}{\sum_jx^{Nj}}\|_F \\
\end{bmatrix},
$$
- The second problem is enforcing the non-negativity constraint, which might imply additional transformations such as re-scaling and translating
- A more serious problem is given by the different normalities of the datasets (binary data, binomial data etc), in which case a more thorough transformation may be necessary (for example via methods such as iCluster+, MOFA, etc)
- Several applications will be discussed later.

In [13]:
from sklearn.decomposition import NMF

nmf = NMF(3)
V = groceries
nmf.fit(V)

H = pd.DataFrame(np.round(nmf.components_,2), columns=V.columns)
H.index = latent_factors

W = pd.DataFrame(np.round(nmf.transform(V),2), columns=H.index)
W.index = V.index

print("\n\nV Data matrix (features x samples):")
print(V)
print("\n\nW factors matrix (features, factors):")
print(W)
print("\n\nH coeficients matrix (factors, samples):")
print(H)



V Data matrix (features x samples):
          John  Alice  Mary  Greg  Peter  Jennifer
Potatoes     0      1     0     1      2         2
Onions       0      1     1     1      3         4
Fruits       2      3     1     1      2         2
Sweets       1      1     1     0      1         1
Bread        0      2     3     4      1         1
Coffee       0      0     0     0      1         0


W factors matrix (features, factors):
          Fruits pickers  Bread eaters  Veggies
Potatoes            1.03          0.35     0.08
Onions              1.80          0.56     0.00
Fruits              0.94          0.17     1.51
Sweets              0.46          0.01     0.67
Bread               0.47          2.41     0.00
Coffee              0.23          0.00     0.00


H coeficients matrix (factors, samples):
                John  Alice  Mary  Greg  Peter  Jennifer
Fruits pickers  0.00   0.37  0.03  0.12   1.74      2.13
Bread eaters    0.00   0.76  1.22  1.64   0.06      0.00
Veggies        

/media/sergiu/workpc/programs/miniconda3/envs/iocourse/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/media/sergiu/workpc/programs/miniconda3/envs/iocourse/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


### Why is NMF so sexy?

Because it provides an intuitive interpretation of the results, due to its enforcement of the positive values rule.
- The higher the factor weight, the more “determined” the column (segment) is by the variable in the row.
- First components are more "determined" based on the data.

Observations:
- It is now clear that veggies eaters are not described by the third component in the latent space. 
    - the correct latent vector is in fact ['Veggies', 'Fruits pickers', 'Bread eaters']
    - the components of a latent vectors are also called "segments", this name comes from the initial use of NMF in image segmentation.
- Now we know that Alice likes fruity stuff, but how do we reccomend things?
    - W x H is an approximation of V, so by transforming the dataset based on the NMF model we can learn some new things.
    - The model also learned Jennifer too might enjoy a coffee. This is because she and Peter are both "Veggies", and Peter once bought a coffee. Plus, she hates bread and fruits/sugars and so she must have a vice. :) 
    - This is an example of collaborative filtering for marketing. During the lab, you too can now recomend interesting gene therapies to cancer patients, based on their hidden subtypes!

Reference: I drew inspiration from the following Medium article:
- https://medium.com/logicai/non-negative-matrix-factorization-for-recommendation-systems-985ca8d5c16c


In [14]:
reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=V.columns)
reconstructed.index = V.index

print(reconstructed)

          John  Alice  Mary  Greg  Peter  Jennifer
Potatoes  0.11   0.78  0.51  0.72   1.83      2.19
Onions    0.00   1.09  0.74  1.13   3.17      3.83
Fruits    2.04   2.89  1.20  0.87   2.02      2.00
Sweets    0.90   1.25  0.45  0.29   0.97      0.98
Bread     0.00   2.01  2.95  4.01   0.96      1.00
Coffee    0.00   0.09  0.01  0.03   0.40      0.49


Further read:
- NMF outperforms similar techniques at learning parts representation:
    - Lee, D., Seung, H. Learning the parts of objects by non-negative matrix factorization. Nature 401, 788–791 (1999). https://doi.org/10.1038/44565


  - NMF was probably the best hidden feature based image segmentation prior to CNNs
  - In comparison to other MF methods (especially the orthogonality enforcing methods), NMF hidden features are not independent, but overlapping, in a hierarchical manner (thus good for hierarchical clustering)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

V = np.array([[0,1,0,1,2,2],
              [2,3,1,1,2,2],
              [1,1,1,0,1,1],
              [0,2,3,4,1,1],
              [0,0,0,0,1,0]])

V = pd.DataFrame(V, columns=['John', 'Alice', 'Mary', 'Greg', 'Peter', 'Jennifer'])
V.index = ['Vegetables', 'Fruits', 'Sweets', 'Bread', 'Coffee']

nmf = NMF(3)
nmf.fit(V)

H = pd.DataFrame(np.round(nmf.components_,2), columns=V.columns)
H.index = ['Fruits pickers', 'Bread eaters',  'Veggies']

W = pd.DataFrame(np.round(nmf.transform(V),2), columns=H.index)
W.index = V.index

reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=V.columns)
reconstructed.index = V.index

## NMF - algorithms

- Chen et al, Attention-Based Multi-NMF Deep Neural Network with Multimodality Data for Breast Cancer Prognosis Model, Volume 2019 |Article ID 9523719 | https://doi.org/10.1155/2019/9523719


- multiplicative update algorithms
- Alternating Least Square
- NMF based on Optimal Brain Surgery and Alternate Least Square algorithms
- NMF based on projection gradient algorithms
- PNMF probabilistic nonnegative matrix factorization



# SNMF - Sparse NMF

- Yuan Gao, George Church, Improving molecular cancer class discovery through sparse non-negative matrix factorization, Bioinformatics, Volume 21, Issue 21, , Pages 3970–3975, https://doi.org/10.1093/bioinformatics/bti653
- Brunet et al., 2004, Metagenes and molecular pattern discovery using matrix factorization
PNAS, Mar 2004, 101 (12) 4164-4169; https://doi.org/10.1073/pnas.0308531101


- cancer subtyping in microarrays, dealing with sparse data
- not integrative (but does that *really* matter?)
- rank estimation of NMF models
- the representation learned is sparse and hierarchical
- $A \approx WH$ The more sparse the matrix of H, the more sparse is the feature matrix W. Therefore, enforcing the sparseness of H will give rise to metagenes that comprised few dominantly deterministic genes.

# jNMF

- Fujita, N., Mizuarai, S., Murakami, K. et al. Biomarker discovery by integrated joint non-negative matrix factorization and pathway signature analyses. Sci Rep 8, 9743 (2018). https://doi.org/10.1038/s41598-018-28066-w

Main goals:
- biomarker discovery
- is theoretically and practically equivalent to a standard NMF method with concatenated inputs
- common clusters (co-modules) from mRNA expression, microRNA expression, and DNA methylation data of cancer patients

JNMF - method details

- objective function: $\sum_{i=1}^N \|X_i - WH_i\|$
- NMF method was modified to deal with missingness, by using a mask

<img src="img/jnmf_pharma.png" width="500"/>


### iNMF

- Zi Yang and George Michailidis, A non-negative matrix factorization method for detecting modules in heterogeneous omics multi-modal data, https://doi.org/10.1093/bioinformatics/btv544
- similar to jNMF, code: https://github.com/yangzi4/iNMF

- Used recently in a method for online learning of integrative omics for single cell:
    - Gao, C., Liu, J., Kriebel, A.R. et al. Iterative single-cell multi-omic integration using online learning. Nat Biotechnol (2021). https://doi.org/10.1038/s41587-021-00867-x

## Deep MF

- Different NN setups exist for almost every type of MF!
- Example: Supervised deep matrix factorization on the MovieLens dataset, using mxnet:
    - https://www.oreilly.com/content/deep-matrix-factorization-using-apache-mxnet/
- Observation: MF can also be formulated via autoencoders.

Advantages:
- Take advantage of NN advancements such as mixing batch normalization layers to speed up training time.
- Restrict the size of user/item factors independently. Useful in the case where one dimension may be significantly larger than the other and, thus, requires training a massive number of factors

<img src="img/deep_mf.png" width="300"/>


In [ ]:
X_train = mx.io.NDArrayIter({'user': train_users, 'movie': train_movies, 'movie_genre': train_genres}, 
                            label=train_ratings, batch_size=batch_size)
X_eval = mx.io.NDArrayIter({'user': valid_users, 'movie': valid_movies, 'movie_genre': valid_genres}, 
                           label=valid_ratings, batch_size=batch_size)

user = mx.symbol.Variable("user")
user = mx.symbol.Embedding(data=user, input_dim=n_users, output_dim=15)

movie = mx.symbol.Variable("movie")
movie = mx.symbol.Embedding(data=movie, input_dim=n_movies, output_dim=20) # Reduce from 25 to 20

# We need to add in a third embedding layer for genre
movie_genre = mx.symbol.Variable("movie_genre")
movie_genre = mx.symbol.Embedding(data=movie_genre, input_dim=20, output_dim=5) # Set to 5

y_true = mx.symbol.Variable("softmax_label")

nn = mx.symbol.concat(user, movie, movie_genre)
nn = mx.symbol.flatten(nn)
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=64)
nn = mx.symbol.Activation(data=nn, act_type='relu')
nn = mx.symbol.FullyConnected(data=nn, num_hidden=1)

y_pred = mx.symbol.LinearRegressionOutput(data=nn, label=y_true)

model = mx.module.Module(context=mx.gpu(0), data_names=('user', 'movie', 'movie_genre'), symbol=y_pred)
model.fit(X_train, num_epoch=5, optimizer='adam', optimizer_params=(('learning_rate', 0.001),),
          eval_metric='rmse', eval_data=X_eval, batch_end_callback=mx.callback.Speedometer(batch_size, 250))

## Deep NMF

- Flenner, J., & Hunter, B. (2017). A Deep Non-Negative Matrix Factorization Neural Network.
    - https://www1.cmc.edu/pages/faculty/BHunter/papers/deep-negative-matrix.pdf


- Normal NMF is performed by a single encoding layer 

<img src="img/nmf_onelayer.png" width="200"/>




- Deep architecture: CNN, with backpropagation, each NMF layer performs a hierarchical decomposition

<img src="img/nmf_multilayered.png" width="200"/>


## Matrix Factorization - beyond NMF

- Scluster method (a mix of MF and SNF): integrates different types of data and maps them into an effective low-dimensional subspace. First, Scluster uses adaptive sparse reduced-rank regression (S-rrr) to map the original data into the principal subspaces. Next, a fused patient-by-patient network is abstracted for these subgroups by a scaled exponential similarity kernel method. It can then obtain the cancer subtypes by spectral clustering.
    - https://pubmed.ncbi.nlm.nih.gov/28113782/
- SRF : rank based multi-view bi-clustering, very related to NMF, possibly reducible to it, it does subtyping and identification of subtype-specific features simultaneously.
    - https://pubmed.ncbi.nlm.nih.gov/27587661/

Software:
- Python:
    - recommender systems library: http://surpriselib.com/
    - NMF library: http://nimfa.biolab.si/
    - https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
- R:
    - general NMF: http://cran.r-project.org/package=NMF
    - usage example: https://compgenomr.github.io/book/biological-interpretation-of-latent-factors.html
    - joint NMF library: https://github.com/rikenbit/nnTensor/
- Deep Semi NMF: https://github.com/trigeorgis/Deep-Semi-NMF


Rank estimation of NMF:
- Jean-Philippe Brunet. et. al., (2004). Metagenes and molecular pattern discovery using matrix factorization. PNAS
- Xiaoxu Han. (2007). CANCER MOLECULAR PATTERN DISCOVERY BY SUBSPACE CONSENSUS KERNEL CLASSIFICATION
- Attila Frigyesi. et. al., (2008). Non-Negative Matrix Factorization for the Analysis of Complex Gene Expression Data: Identification of Clinically Relevant Tumor Subtypes. Cancer Informatics
- Haesun Park. et. al., (2019). Lecture 3: Nonnegative Matrix Factorization: Algorithms and Applications. SIAM Gene Golub Summer School, Aussois France, June 18, 2019
- Chunxuan Shao. et. al., (2017). Robust classification of single-cell transcriptome data by nonnegative matrix factorization. Bioinformatics


Other integrative NMF:


- intNMF: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0176278
- Simultaneous Non-negative Matrix Factorization (siNMF) : Extracting Gene Expression Profiles Common to Colon and Pancreatic Adenocarcinoma using Simultaneous nonnegative matrix factorization, Liviu Badea, Pacific Symposium on Biocomputing, 13:279-290, 2009, 
- Discovery of multi-dimensional modules by integrative analysis of cancer genomic data. Shihua Zhang, et al., Nucleic Acids Research, 40(19), 9379-9391, 2012
- Probabilistic Latent Tensor Factorization, International Conference on Latent Variable Analysis and Signal Separation, Y. Kenan Yilmaz et al., 346-353, 2010


Fast Tensorial Calculus:


- Non-negative Matrix Factorization (NMF) : [Nonnegative Matrix and Tensor Factorizations, Andrzej CICHOCK, et. al., 2009](https://pdfs.semanticscholar.org/94cc/6daad548a03c6edb0351d686c2d4aa364634.pdf), 
- [A Study on Efficient Algorithms for Nonnegative Matrix/Tensor Factorization, Keigo Kimura, 2017](https://eprints.lib.hokudai.ac.jp/dspace/bitstream/2115/65379/1/Keigo_Kimura.pdf)

Non-negative CP Decomposition (NTF)
   - α-Divergence (KL, Pearson, Hellinger, Neyman) / β-Divergence (KL, Frobenius, IS) : [Non-negative Tensor Factorization using Alpha and Beta Divergence, Andrzej CICHOCKI et. al., 2007](http://mlg.postech.ac.kr/static/publications/inter_conf/2007/icassp07_cichocki.pdf), [TensorKPD.R (gist of mathieubray)](https://gist.github.com/mathieubray/d83ce9c13fcb60f723f957c13ad85ac5)
   - Fast HALS : [Multi-way Nonnegative Tensor Factorization Using Fast Hierarchical Alternating Least Squares Algorithm (HALS), Anh Huy PHAN et. al., 2008](http://www.ieice.org/proceedings/NOLTA2008/articles/A1L-D3-Phan-2045.pdf)
   - α-HALS/β-HALS : [Fast Local Algorithms for Large Scale Nonnegative Matrix and Tensor Factorizations, Andrzej CICHOCKI et. al., 2008](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.214.6398&rep=rep1&type=pdf)
- Non-negative Tucker Decomposition (NTD)
   - KL, Frobenius : [Nonnegative Tucker Decomposition, Yong-Deok Kim et. al., 2007](https://pdfs.semanticscholar.org/f388/99be8ebd8b9aa7029b2b4f187dac4b04d816.pdf)
   - α-Divergence (KL, Pearson, Hellinger, Neyman) / β-Divergence (KL, Frobenius, IS) : [Nonneegative Tucker Decomposition With Alpha-Divergence, Yong-Deok Kim et. al., 2008](https://pdfs.semanticscholar.org/f01b/7354619f053863048217c58cc517def86aeb.pdf), [Fast and efficient algorithms for nonnegative Tucker decomposition, Anh Huy Phan, 2008](https://link.springer.com/chapter/10.1007/978-3-540-87734-9_88)
   - Fast HALS : [Extended HALS algorithm for nonnegative Tucker decomposition and its applications for multiway analysis and classification, Anh Hyu Phan et. al., 2011](https://www.sciencedirect.com/science/article/pii/S0925231211000427)